# Индивидуальный проект

- Название проекта:
- Вариант задания
- Описание проекта:

[ваш текст]

**Дополнительное задание**

Добавьте к существующим классам (базовыму и производным 3-4 атрибута и метода) используйте в проекте коллекции, делегаты, события.


In [ ]:
using System;
using System.Collections.Generic;

public class Order
{
    public int OrderId { get; set; } // поля класса
    public DateTime CreationDate { get; set; }
    public decimal TotalAmount { get; set; }
    protected List<Item> Items { get; set; } // позволяет хранить упорядоченные элементы

    public Order(int orderId) // констр
    {
        OrderId = orderId;
        CreationDate = DateTime.Now;
        Items = new List<Item>();
        TotalAmount = 0;
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in Items)
        {
            TotalAmount += item.Price;
        }
    }

    public virtual void AddItem(Item item)
    {
        Items.Add(item);
        CalculateTotal();
    }

    public virtual void RemoveItem(Item item)
    {
        if (Items.Remove(item))
        {
            CalculateTotal();
        }
    }

    public virtual void PrintDetails()
    {
        Console.WriteLine($"Заказ ID: {OrderId}, Дата создания: {CreationDate}, Общая сумма: {TotalAmount} р.");
    }

    
    
}


public delegate void DisplayDelegate();

public class Item // дополнительный класс
{
    public string Name { get; set; }
    public decimal Price { get; set; }

    public Item(string name, decimal price) // констр
    {
        Name = name;
        Price = price;
    }
    public virtual void GetInfo()
    {
        Console.WriteLine($"Товар: {Name}. Цена: {Price}");
    }

    public override bool Equals(object obj)
    {
        // Простая проверка равенства базируется на названии и цене
        if (obj is Item item)
        {
            return Name == item.Name && Price == item.Price;
        }
        return false;
    }

    public override int GetHashCode()
    {
        // Создание хеш-кода на основе названия и цены
        return HashCode.Combine(Name, Price);
    }
    public delegate void ItemHandler(string message);
    public event ItemHandler PriceChanged;

    public void ChangedPrice(string name, int newPrice) //событие
    {
        Price = newPrice;
        // вызываем событие после обновления цены
        PriceChanged?.Invoke($"Цена товара {name} изменена на {newPrice}");
    }
}

public class OnlineOrder : Order// обобщенный класс
{
    public string CustomerEmail { get; set; }
    public OnlineOrder(int orderId, string customerEmail) : base(orderId)
    {
        CustomerEmail = customerEmail;
    }
    public override void AddItem(Item item)
    {
        base.AddItem(item);
        PrintAdd(item);
    }
    public override void RemoveItem(Item item)
    {
        base.RemoveItem(item);
        PrintRemove(item);
    }
    private void PrintAdd(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' добавлен в онлайн-заказ для {CustomerEmail}");
    }
    private void PrintRemove(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' удален из онлайн-заказа.");
    }
    public override void PrintDetails()
    {
        base.PrintDetails();
        Console.WriteLine($"Email клиента: {CustomerEmail}");
    }
}

public class PhysicalOrder : OnlineOrder // сложное наследование
{
    public string DeliveryAddress { get; set; }
    public PhysicalOrder(int orderId, string deliveryAddress) : base(orderId, "---")
    {
        DeliveryAddress = deliveryAddress;
    }
    public override void PrintDetails()
    {
        base.PrintDetails();
        Console.WriteLine($"Адрес доставки: {DeliveryAddress}");
    }
}

public interface ISpecializedOrder // интерфейс
{
    void SetSpecialConditions(string specialConditions);
    void Sale(Item item);
}

public class SpecializedOrder : Order, ISpecializedOrder //  множ наследование
{
    public string SpecialConditions { get; private set; }
    public bool HasSpecialConditions { get; private set; } // булева переменная
    public SpecializedOrder(int orderId, string specialConditions) : base(orderId)
    {
        SetSpecialConditions(specialConditions);
    }
    public void SetSpecialConditions(string specialConditions)
    {
        SpecialConditions = specialConditions;
        HasSpecialConditions = !string.IsNullOrEmpty(specialConditions); // проверка на пустоту или null
    }
    public void Sale(Item item)
    {
        Console.WriteLine($"На товар '{item.Name}' действует скидка 10%!");
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        if (HasSpecialConditions) 
        {
            TotalAmount *= 0.9m; 
        }
    }

    public override void PrintDetails()
    {
        base.PrintDetails();
        if (HasSpecialConditions)
        {
            Console.WriteLine($"Специальные условия: {SpecialConditions}");
        }
        else
        {
            Console.WriteLine("Специальные условия отсутствуют.");
        }
    }
}

// лист
List<Item> itemList = new List<Item>();

itemList.Add(new Item("Книга", 150));
itemList.Add(new Item("Игрушка", 200));

Console.Write("List:\n");
foreach (var item in itemList)
{
    item.GetInfo();
}

Console.WriteLine();

// дикшинари
Dictionary<string, Item> itemDictionary = new Dictionary<string, Item>();

itemDictionary.Add("1", new Item("Стол", 1500));
itemDictionary.Add("2", new Item("Стул", 800));

Console.Write("Dictionary:\n");
if (itemDictionary.TryGetValue("1", out Item foundItem))
{
    Console.WriteLine("Найденый товар:");
    foundItem.GetInfo();
}
else
{
    Console.WriteLine("Товар не найден");
}
Console.WriteLine();

//hashset
HashSet<Item> orderSet = new HashSet<Item>();

var item1 = new Item("Кресло", 500);
var item2 = new Item("Подушка", 600);
var item3 = new Item("Сумка", 1400);

orderSet.Add(item1);
orderSet.Add(item2);
orderSet.Add(item3);

// Попробуем добавить повторяющийся объект
var duplicateItem = new Item("Подушка", 600);
bool isAdded = orderSet.Add(duplicateItem);


Console.Write("HashSet:\n");
Console.WriteLine("Список:");
// Перебираем и выводим информацию обо всех уникальных товарах
foreach (var item in orderSet)
{
    item.GetInfo();
}

Console.WriteLine();
// Присваиваем делегату метод DisplayInfo
DisplayDelegate displayItem1 = item1.GetInfo;
DisplayDelegate displayItem2 = item2.GetInfo;
Console.WriteLine("Делегаты");
// Вызов методов через делегаты
displayItem1();
displayItem2();

//лямбда
Console.WriteLine();
Console.WriteLine("Лямбда:");
var book = itemList.OfType<Item>().Where(c => c.Name == "Книга");

foreach (var item in book)
{
    item.GetInfo();
}

//события
Console.WriteLine();
Console.WriteLine("События");

item1.PriceChanged += OnPriceChanged;
// Обновляем цену; это вызовет событие
item1.ChangedPrice(item1.Name, 300);

static void OnPriceChanged(string message)
{
    Console.WriteLine(message);
}


List:
Товар: Книга. Цена: 150
Товар: Игрушка. Цена: 200

Dictionary:
Найденый товар:
Товар: Стол. Цена: 1500

HashSet:
Список:
Товар: Кресло. Цена: 500
Товар: Подушка. Цена: 600
Товар: Сумка. Цена: 1400

Делегаты
Товар: Кресло. Цена: 500
Товар: Подушка. Цена: 600

Лямбда:
Товар: Книга. Цена: 150

События
Цена товара Кресло изменена на 300
